# RiCA Demo Notebook
This notebook demonstrates the basic functionality of the RiCA framework.

In [ ]:
# Run this line if you're running this notebook in Google Colab or similar environments
# Choose ONE of the following options:
# Option 1: Install from PyPI
# !pip install rica-server[pt]

# Option 2: Install from repository
# !git clone https://github.com/rica-team/rica-server.git

!pip install .[pt]

In [ ]:
from huggingface_hub import login
login()

In [ ]:
import json
import logging
import ast
from typing import Any
from rica.adapters import transformers_adapter as tf
from rica.core.application import RiCA

In [ ]:
# Configure logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

# 1. Create RiCA application instance.
app = RiCA("demo.sys", description="System utilities for demo purposes")

In [ ]:
# 2. Register tool routes with the application instance.
@app.route("/exec/", background=False, timeout=5000)
async def _sys_python_exec(input_):
    """
    A tool to execute a single line of Python code safely.
    
    Input Schema:
    {
        "code": "string - Python expression to evaluate"
    }
    
    Output Schema:
    {
        "result": "string - Evaluation result",
        "error": "string - Error message (if any)"
    }
    """
    try:
        code = input_.get("code", "")
        if not code:
            return {"error": "No code provided"}
        
        if len(code) > 100:
            return {"error": "Code too long (max 100 chars)"}
        
        result = ast.literal_eval(code)
        return {"result": str(result)}
    except (ValueError, SyntaxError) as e:
        return {"error": f"Invalid expression: {str(e)}"}
    except Exception as e:
        logger.error(f"Execution error: {e}", exc_info=True)
        return {"error": f"Execution failed: {str(e)}"}

In [ ]:
# 3. Create the Executor.
rt = tf.ReasoningThread(model_name="Qwen/Qwen3-0.6B")

In [ ]:
# 4. Register callbacks.
@rt.token_generated
def _on_token(token: str):
    print(token, end="", flush=True)

@rt.trigger
def _on_response(response_payload: Any):
    print("\n\n--- USER-FACING RESPONSE ---")
    if isinstance(response_payload, list):
        for item in response_payload:
            if item.get("type") == "text":
                print(item.get("content", ""))
    else:
        print(json.dumps(response_payload, indent=2))
    print("----------------------------\n")

In [ ]:
async def main():
    print("--- Starting RiCA Demo ---")

    try:
        await rt.initialize()
        await rt.install(app)

        # Start the executor (if rt.run() is non-blocking, otherwise remove it)
        # rt.run()

        await rt.insert(
            'Please calculate 123*456 using demo.sys/exec. '
            'Think step-by-step about what you need to do, and then use the rica.response tool to give me the final answer.'
        )

        await rt.wait()

        print("\n\n--- Final Context ---")
        print(rt.context)
        print("---------------------")

    except Exception as e:
        logger.error(f"Error in main execution: {e}", exc_info=True)
        raise
    finally:
        await rt.destroy()

In [ ]:
await main()